In [16]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.stats.multitest as smm

In [2]:
def write_answer(filename, value):
    with open(filename, "w") as fout:
        fout.write(str(value))

In [3]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')

In [4]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


### Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)
для групп early neoplasia (control) и cancer (treatment)
В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [14]:
test = data[{'Diagnosis', 'LOC643837'}]
data.loc[data.Diagnosis == 'normal', {'Diagnosis', 'LOC643837'}]

,Diagnosis,LOC643837
0,normal,1.257614
1,normal,4.567931
2,normal,2.077597
3,normal,2.066576
4,normal,2.613616
5,normal,3.942275
6,normal,1.084113
7,normal,3.153900
8,normal,2.551800
9,normal,3.693128


In [6]:
test.head()
columns = pd.Series(data.columns)
columns.pop(0)
columns.pop(1)

'Diagnosis'

In [92]:
def calculate_fold_change(C, T):
    if T>C:
        return T/C
    else:
        return -C/T
    
normal_early_fold = np.array([])
early_cancer_fold = np.array([])
normal_early_t_p_value = np.array([])
early_cancer_t_p_value = np.array([])

#  normal_early_holm_p_value, early_cancer_holm_p_value,
#  normal_early_b_h_p_value, early_cancer_b_h_p_value = np.array()

for column in columns:
    normal_mean = data.loc[data.Diagnosis == 'normal', column].mean()
    early_mean = data.loc[data.Diagnosis == 'early neoplasia', column].mean()
    cancer_mean = data.loc[data.Diagnosis == 'cancer', column].mean()
    
    normal_early_fold = np.append(normal_early_fold, calculate_fold_change(normal_mean, early_mean))
    early_cancer_fold = np.append(early_cancer_fold, calculate_fold_change(early_mean, cancer_mean))
    
    normal_early_t_p_value = np.append(normal_early_t_p_value, stats.ttest_ind(data.loc[data.Diagnosis == 'normal', column],
                                                                             data.loc[data.Diagnosis == 'early neoplasia', column],
                                                                             equal_var=False).pvalue)
    
    
    early_cancer_t_p_value = np.append(early_cancer_t_p_value, stats.ttest_ind(data.loc[data.Diagnosis == 'early neoplasia', column],
                                                                                     data.loc[data.Diagnosis == 'cancer', column],
                                                                                     equal_var=False
                                                                                    ).pvalue)

    
    


In [93]:

print(np.count_nonzero(normal_early_t_p_value < 0.05))
print(np.count_nonzero(early_cancer_t_p_value < 0.05))

1575
3490


### Часть 2: поправка методом Холма



In [101]:
normal_early_holm_p_value = smm.multipletests(pvals=normal_early_t_p_value,
                                              method='holm')[1]
early_cancer_holm_p_value = smm.multipletests(pvals=early_cancer_t_p_value,
                                              method='holm')[1]

In [102]:
print(np.count_nonzero(normal_early_holm_p_value < 0.025))
print(np.count_nonzero(early_cancer_t_p_value < 0.025))

2
2611


In [132]:
normal_early_holm = pd.DataFrame(np.transpose(np.vstack((normal_early_fold, normal_early_holm_p_value))),
                                columns = ['c', 'p'])
early_cancer_holm = pd.DataFrame(np.transpose(np.vstack((early_cancer_fold, early_cancer_holm_p_value))),
                                columns = ['c', 'p'])
normal_early_holm.head()

,c,p
0,-1.067858,1.000000
1,1.996517,0.500174
2,1.222418,1.000000
3,-1.020070,1.000000
4,1.125471,1.000000


In [133]:
normal_early_holm[(normal_early_holm.p < 0.025) & (abs(normal_early_holm.c) > 1.5)].count()
early_cancer_holm[(early_cancer_holm.p < 0.025) & (abs(early_cancer_holm.c) > 1.5)].count()

c    77
p    77
dtype: int64

In [135]:
write_answer('question1.txt', 2)
write_answer('question2.txt', 77)

### Часть 2: поправка методом Бенджамини-Хохберга



In [137]:
normal_early_b_h_p_value = smm.multipletests(pvals=normal_early_t_p_value,
                                              method='fdr_bh')[1]
early_cancer_b_h_p_value = smm.multipletests(pvals=early_cancer_t_p_value,
                                              method='fdr_bh')[1]

In [138]:
normal_early_fdr_bh = pd.DataFrame(np.transpose(np.vstack((normal_early_fold, normal_early_b_h_p_value))),
                                columns = ['c', 'p'])
early_cancer_fdr_bh = pd.DataFrame(np.transpose(np.vstack((early_cancer_fold, early_cancer_b_h_p_value))),
                                columns = ['c', 'p'])
normal_early_holm.head()

,c,p
0,-1.067858,1.000000
1,1.996517,0.500174
2,1.222418,1.000000
3,-1.020070,1.000000
4,1.125471,1.000000


In [145]:
print(normal_early_fdr_bh[(normal_early_fdr_bh.p < 0.025) & (abs(normal_early_fdr_bh.c) > 1.5)].count())
print(early_cancer_fdr_bh[(early_cancer_fdr_bh.p < 0.025) & (abs(early_cancer_fdr_bh.c) > 1.5)].count())

c    4
p    4
dtype: int64
c    524
p    524
dtype: int64


In [146]:
write_answer('question1.txt', 4)
write_answer('question2.txt', 524)